In [ ]:
import mlflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from datetime import datetime, timedelta
from pydash import py_
from green_city.utils import datetime2index, index2datetime

from green_city.mlflow_config import get_mlflow_config
mlflow.set_tracking_uri(get_mlflow_config()["TRACKING_URI"])
client = MlflowClient()
experiment_id = client.get_experiment_by_name("green_city_experiments").experiment_id

## DB CONNECTION ##
from sqlalchemy import create_engine
from decouple import Config, RepositoryEnv

config = Config(RepositoryEnv("../.db_credentials"))

db_connection_credentials = {
    "database": config('POSTGRES_DB'),
    "user": config('POSTGRES_USER'),
    "password": config('POSTGRES_PASSWORD'),
    "host": config('POSTGRES_HOST'),
    "port": config('POSTGRES_PORT'),
}
DB_STRING = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**db_connection_credentials)
db = create_engine(DB_STRING)

In [ ]:
run_ids = [r.run_id for r in client.list_run_infos(experiment_id, run_view_type=ViewType.ACTIVE_ONLY)]

all_runs = {run_id: client.get_run(run_id).to_dictionary() for run_id in run_ids}

In [ ]:
runs_dict = {}

for id, details in all_runs.items():
    start_time = py_.get(details, "info.start_time")
    end_time = py_.get(details, "info.end_time")
    #delta = round(end_time/1000 - start_time/1000, 2)
    runs_dict[id] = {
        #"time_s": delta,
        **py_.get(details, "data.metrics"),
        **py_.get(details, "data.params"),
        "run_name": details["data"]["tags"]["mlflow.runName"],
    }

In [ ]:
df = pd.DataFrame.from_dict(runs_dict, orient='index')
df = df.astype({"datetime": "datetime64[ns]"})
df = df.assign(index = df['index'].fillna(df.datetime.map(datetime2index))).dropna(subset=["index"])

#if index is not set, assign it from datetime
df.datetime.map(datetime2index).unique()
df = df.astype({"index": "int", "train_steps": "float"})
df.index = df.index.rename("run_id")

#Drop the shorter TBATS models
df = df[np.logical_or(np.logical_not(df.train_steps<1000), df.model!="TBATS")]
df = df[df.model!="ARIMA"]

In [ ]:
#df[df["run_name"] == "Prophet Simple Model"].index

In [ ]:
#plt.rcParams['figure.figsize'] = [25, 8]
#df[df.feature=="net_load_kWh"].loc[:, ["mae", "mse", "r2_score"]].plot.barh()
#df[df.feature=="net_load_kWh"].loc[:, ["time_s"]].plot.barh()
df.to_csv("../data/results/evaluation_matrix.csv")

In [ ]:
def row_label(df_row):
    if df_row["run_name"] == "Prophet Simple Model":
        return "Prophet"
    
    if df_row["model"] == "TBATS":
        return df_row['model']
        #if not np.isnan(df_row["train_steps"]):
        #    return df_row['model'] + " (" + str(int(df_row['train_steps'])) + " trainsteps)"
        #else:
        #    return df_row['model']
    elif df_row["model"] == "sarimax":
        return "Sarimax"
        # return df_row["run_name"].capitalize().replace("_", " ")
    elif df_row["model"] == "baseline":
        return "Baseline"
    elif df_row["model"] == "linear_regression":
        return "Linear regression"
    else:
        return df_row["model"]
    
df.loc[df["run_name"] == "Prophet Simple Model", "model"] = "prophet"
df.loc[df["run_name"] == "Prophet Simple Model Optimized Parameters", "model"] = "prophet"


df["disp_label"] = df.apply(row_label, axis=1)


In [ ]:
plt.rcParams['figure.figsize'] = [12, 4]
USE_ROOT = True
   
model2color = {
    "baseline": "orange",
    "linear_regression": "gray",
    "TBATS": "blue",
    "sarimax": "green",
    "ARIMA": "purple",
    "prophet": "purple",
}

all_performances = {
    "baseline": [],
    "linear_regression": [],
    "TBATS": [],
    "sarimax": [],
    "ARIMA": [],
    "prophet": [],
}

indices = [32135, 33311, 26478, 33357, 30387, 30794, 31800, 28783]
for index in indices:
    fig, ax = plt.subplots()
    fig.set_figheight(2)
    fig.set_figwidth(5)
    fig.set_dpi(240)

    plot_df = (df[df["index"] == index]
                .sort_values("mse", ascending=False)
                .assign(col = lambda x:x["model"].map(lambda x: model2color.get(x, "black")))
                #.assign(label = lambda x: f"{x['model']}{steps2str(x['train_steps'])}")
                .drop_duplicates(subset=["run_name"])
    )
    plot_df["rmse"] = np.sqrt(plot_df["mse"])
    
    plot_df = plot_df[plot_df["run_name"] != "sarimax_(6,1,1)(0,0,2,24)"]
    plot_df = plot_df[plot_df["disp_label"].map(lambda x: x in ("Linear regression", "Sarimax", "Prophet", "TBATS", "ARIMA", "Baseline"))]
    
    for idx, benchmark in plot_df.iterrows():
        all_performances[benchmark["model"]].append(benchmark["mse"])
    
    if USE_ROOT:
        score_col = "rmse"
    else:
        score_col = "mse"
    plot_df.plot.barh(x="disp_label", y=score_col, color=plot_df["col"], legend=None, ax=ax)
    #ax.barh(plot_df["mse"], width=20, color=plot_df["col"], legend=None, height=0.9)
    axis_label = "Root Mean Squared Error" if USE_ROOT else "Mean squared error"
    ax.set_xlabel(axis_label)
    ax.set_ylabel(None)
    #fig.suptitle(f"{str(index2datetime(index+1))} to {str(index2datetime(index+24))}")
    fig.suptitle("")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

In [ ]:
avg_performance = {}
for k, v in all_performances.items():
    if k == "ARIMA":
        continue
    #print(k, v)
    #avg_performance[k] = [0]
    avg_performance[k] = np.mean(v)
display(avg_performance)
mmses = pd.concat([pd.DataFrame({"model": [k], "mmse": [v]}) for k, v in avg_performance.items()], axis=0)
mmses = mmses.sort_values("mmse", ascending=False)
mmses["color"] = mmses.model.map(lambda x: model2color.get(x, "black"))

fig, ax = plt.subplots()
fig.set_figheight(4)
fig.set_figwidth(9)
fig.set_dpi(240)

mmses.plot.barh(x="model", y="mmse", color=mmses["color"], ax=ax)
ax.set_xlabel("Average performance (Mean Squared Error)")
ax.set_ylabel("")
ax.legend().remove()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)

In [ ]:
# Load the actual dataset
def get_df(building_nr, feature):
    df = (
    pd.read_csv(f"../data/preprocessed/Building_{building_nr}.csv")
        .astype({'datetime': 'datetime64'})
        [[feature, "datetime"]]
        .rename(columns={feature: "actual"})
    )
    df.index.name = "id"
    return df

actual_df = get_df(5, "net_load_kWh")

In [ ]:
plt.rcParams['figure.figsize'] = [25, 6]

index_to_plot = indices[7]
#index_to_plot = indices[1]
plot_df = df[df["index"] == index_to_plot].drop_duplicates(subset=["run_name"])
plot_df

In [ ]:
from pandas import concat


df_from_csv_1 = pd.read_csv("../data/results/linear_regression.csv")
df_from_csv_2 = pd.read_csv("../data/results/sarimax_(2,1,1)(1,0,1,24).csv")
df_from_csv = pd.concat([df_from_csv_1, df_from_csv_2], axis=0, ignore_index=True)[["runid", "id", "prediction"]].rename(columns={"runid": "run_id"})
#df_from_csv.head()

In [ ]:
#def y_fmt(x, y):
#    if max_y > 1000000:
#        val = int(y)/1000000
#        return '{:d} M'.format(val)
#    elif max_y > 1000:
#        val = int(y) / 1000
#        return '{:d} k'.format(val)
#    else:
#        return y
import matplotlib.ticker as tick
from matplotlib.dates import DateFormatter

y_true = actual_df.loc[index_to_plot+1:index_to_plot+24].set_index("datetime")

fig = plt.figure(figsize=(11,3), dpi=240)
ax = fig.add_subplot(111)


for run_id, run_data in plot_df.iterrows():
    plot_kwargs = {}
    query_string = f"SELECT * FROM forecast WHERE run_id='{run_id}'" # LIMIT 10"
    run_predictions = pd.read_sql(query_string, db).drop(columns="run_id")
    if len(run_predictions) == 0:
        #print(f"[INFO: for {run_id}, there is not data in the SQL database.]")
        #print(f"[...trying to read from csv file]")
        run_predictions = df_from_csv[df_from_csv.run_id == run_id].copy().drop(columns="run_id")
        if len(run_predictions) == 0:
            print(f"[Warning: didn't find {run_id} csv data]")
            continue
        if run_data.run_name in ["linear_regression", "sarimax_(2,1,1)(1,0,1,24)"]:
            plot_kwargs = {
                'linestyle': 'dashed'
            }

    run_predictions["datetime"] = run_predictions["id"].map(index2datetime)#y_true.index
    color = model2color.get(run_data.model, "gray")
    ax.plot(run_predictions.drop(columns=["id"]).set_index("datetime"), color=color, label=run_data["disp_label"], **plot_kwargs)
    #ax.plot(run_predictions.drop(columns=["id"]).set_index("datetime"), color=color, label=run_data["disp_label"])
ax.plot(y_true, color="black", label="actual data");
#plt.yaxis.set_major_formatter(tick.FuncFormatter(y_fmt))
ax.legend(prop={'size': 9})

def y_fmt(x, y):
    return f"{int(x)} kW"
    #return f"x:{x}; y:{y}"
    #return '{:2.2e}'.format(x).replace('e', 'x10^')

#ax.set_ylim(21, 82)
ax.set_ylim(0, 62)
ax.yaxis.set_major_formatter(tick.FuncFormatter(y_fmt))
date_form = DateFormatter("%H:%M")
ax.xaxis.set_major_formatter(date_form)
#fig.suptitle("Net energy usage forecasts (Year 4, Sep 2)")
fig.suptitle("Net energy usage forecasts (Year 4, Apr 15 - 16)")
ax.legend(bbox_to_anchor=(1,0), loc="lower left")

In [ ]:
query_string = "SELECT * FROM forecast" # LIMIT 10"
run_predictions = pd.read_sql(query_string, db)

In [ ]:
len(run_predictions)

In [ ]:
indices = [32135, 33311, 26478, 33357, 30387, 30794, 31800, 28783]
times = [(index2datetime(i), i+1, i+24) for i in indices]
display(times)

# upload building data from csv to database

In [ ]:
if False:
    building_nr = 5
    df = pd.read_csv(f"../data/preprocessed/Building_{building_nr}.csv").astype({'datetime': 'datetime64'})
    df.index.name = "id"
    
    df.to_sql("building5", con=db)